In [1]:
import pandas as pd
import implicit
import lightfm
import scipy

import string
# Библиотека построения индекса приближенного поиска ближайших соседей
import annoy
import numpy as np

from pymorphy3 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split

c:\Users\anzel\anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\anzel\anaconda3\envs\py311\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("en")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [3]:
# Загрузим рейтинги, также как делали
ratings = pd.read_csv("data/ml-100k/u.data", sep="\t", header=None)
ratings.columns = ['user id', 'item id', 'rating', 'timestamp']
ratings.sort_values('timestamp', inplace=True)

ratings['score'] = (ratings['rating'] > 2).apply(int)
train, test = train_test_split(ratings, test_size=0.2, shuffle=False)

ratings.head()

,user id,item id,rating,timestamp,score
214,259,255,4,874724710,1
83965,259,286,4,874724727,1
43027,259,298,4,874724754,1
21396,259,185,4,874724781,1
82655,259,173,4,874724843,1


In [4]:
# Загрузим данные о фильмах, сразу же создадим колонку с title+description предобработанные для обучения fasttext
movies = pd.read_csv("data/ml-100k/movies.csv")
movies['joined'] = (movies['title'] + " " + movies["description"]).apply(preprocess_txt)

movies.head()

,id,title,tagline,description,genres,keywords,date,collection,runtime,revenue,...,cast,production_companies,production_countries,popularity,average_vote,num_votes,language,imdb_id,poster_url,joined
0,862,Toy Story,NaN,"Led by Woody, Andy's toys live happily in his ...","animation, comedy, family","jealousy, toy, boy, friendship, friends, rival...",1995-10-30,Toy Story Collection,81.0,373554033.0,...,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",Pixar Animation Studios,United States of America,21.946943,7.7,5415.0,en,tt0114709,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[toy, story, led, woody, andys, toys, live, ha..."
1,8844,Jumanji,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,"adventure, fantasy, family","board game, disappearance, based on children's...",1995-12-15,NaN,104.0,262797249.0,...,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,17.015539,6.9,2413.0,en,tt0113497,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[jumanji, siblings, judy, peter, discover, enc..."
2,15602,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,"romance, comedy","fishing, best friend, duringcreditsstinger, ol...",1995-12-22,Grumpy Old Men Collection,101.0,0.0,...,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","Warner Bros., Lancaster Gate",United States of America,11.712900,6.5,92.0,en,tt0113228,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[grumpier, old, men, family, wedding, reignite..."
3,31357,Waiting to Exhale,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...","comedy, drama, romance","based on novel, interracial relationship, sing...",1995-12-22,NaN,127.0,81452156.0,...,"Whitney Houston, Angela Bassett, Loretta Devin...",Twentieth Century Fox Film Corporation,United States of America,3.859495,6.1,34.0,en,tt0114885,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[waiting, exhale, cheated, mistreated, stepped..."
4,11862,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,comedy,"baby, midlife crisis, confidence, aging, daugh...",1995-02-10,Father of the Bride Collection,106.0,76578911.0,...,"Steve Martin, Diane Keaton, Martin Short, Kimb...","Sandollar Productions, Touchstone Pictures",United States of America,8.387519,5.7,173.0,en,tt0113041,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[father, bride, part, ii, just, george, banks,..."


In [6]:
# Подготовим кандидатогенератор, который будет отдавать фильмы похожие по текстовому описанию на те, которые оценил пользователь
# Обучим Fasttext и заэмбедим фильмы
sentences = [i for i in movies['joined'] if len(i) > 2]
modelFT = FastText(sentences=sentences, min_count=1, window=5)

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_movies = annoy.AnnoyIndex(20 ,'angular')

# Будем хранить соответствия не только id-> фильм, но и фильм-> id, чтобы быстрее находить эмбеддинги айтемов
index_map_movies = {}
reverse_index_map = {}
counter = 0

for i in tqdm_notebook(range(len(movies))):
    n_ft = 0
    index_map_movies[counter] = movies.loc[i, "title"]
    reverse_index_map[movies.loc[i, "id"]] = counter
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in movies.loc[i, "joined"]:
        if word in modelFT:
            vector_ft += modelFT[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_movies.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_movies.build(10)

C:\Users\anzel\AppData\Local\Temp\ipykernel_15652\3572501135.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(movies))):


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
# Создадим метод, который по id фильма будет возвращать похожие на него по описанию фильмы
def recommend(movie_id, num_sampled=10):
    if movie_id not in reverse_index_map:
        return []
    return ft_index_movies.get_nns_by_item(reverse_index_map[movie_id], num_sampled)

In [ ]:
# Сделаем сразу рандомный кандидатогенератор (он нам пригодится для кандидатогенерации по жанрам)
def sample_random(sample_set, num_sampled=10):
    return np.random.choice(sample_set, num_sampled, replace=False)

In [ ]:
# Сделаем кандидатогенератор по последним просмотренным жанрам

movies2genres = {}
genres_dict = {}

for i in range(len(movies)):
    genres = str(movies.loc[i, "genres"]).strip().split(",")
    movies2genres[movies.loc[i, 'id']] = genres
    for genre in genres:
        if genre not in genres_dict:
            genres_dict[genre] = []
        genres_dict[genre].append(movies.loc[i, 'id'])

In [ ]:
# Создадим метод, который по последнему пролайканному пользователем фильму будет возвращать набор кандидатов
# КОнечно, целесообразно включать более полную историю, но все-таки это учебный пример

movies_unique = movies['id'].unique()

def candidate_generator(movie_id):
    candidates_ft = []
    if movie_id is not None:
        candidates_ft = list(recommend(movie_id))
    candidates_random = list(sample_random(movies_unique))

    candidates_genres = []
    for genre in movies2genres.get(movie_id, []):
        candidates_genres += list(sample_random(genres_dict[genre], 5))
    # Отфильтруем входной фильмы (он мог попасть в кандидаты) и уберем дубликаты
    return set([i for i in candidates_ft + candidates_genres + candidates_random if i!= movie_id])

In [ ]:
# Проверим как работает наш кандидатогенератор

candidate_generator(802)

{350,
 1989,
 2641,
 3081,
 5721,
 7348,
 7547,
 10915,
 11109,
 12565,
 19783,
 21246,
 26691,
 29664,
 35646,
 36931,
 37828,
 40669,
 41862,
 45049,
 49131,
 52237,
 54635,
 56151,
 74122,
 175924,
 183855,
 224746,
 285423,
 337758}

In [ ]:
# Подготовим наши ранжирующие факторизационные машины
train_pivot = (1 + ratings.pivot(index="user id", columns="item id", values="score")).fillna(0)
train_pivot.reset_index(inplace=True, drop=False)
train_pivot = scipy.sparse.csr_matrix(train_pivot)

model = lightfm.LightFM(loss='warp')
model.fit(train_pivot, epochs=30)

In [ ]:
# Посмотрим, насколько статичные рекомендации будут хороши: возьмем последний лайк для каждого пользователя в тренировочном сете
item_ids = set(train['item id'].unique())
known_likes = train[train['score'] > 0].groupby('user id')['item id'].unique().to_dict()

# Получаем последние лайки
last_like = train[train['score'] > 0].drop_duplicates(['user id'], keep="last")[["user id", "item id"]].to_dict()
test_grouped = test[test['score'] > 0].groupby('user id')['item id'].unique().to_dict()


map_at10 = 0

for user_id, liked in tqdm_notebook(test_grouped.items()):
    values = set(liked)
    train_likes = set(known_likes.get(user_id, []))
    candidates = list(candidate_generator(last_like.get(user_id, None)))
    
    # Факторизационные машины не умеют работать с айтемами, которых они не видели
    # ПОэтому надо отфильтровать кандидатов
    candidates = [i for i in candidates if i in item_ids]
    
    # Если мы не смогли ничего отскорить, то считаем, что пользователь просто получил нашу выдачу в рандомном порядке
    recommendations = candidates[:min(10, len(candidates))]
    if len(candidates) != 0:
        recommendations = sorted(list(zip(item_ids, model.predict(user_id, candidates))), key=lambda x: x[1], reverse=True)
        recommendations = set([i[0] for i in recommendations if i not in train_likes][:10])
    map_at10 += len(values.intersection(recommendations)) / 10
map_at10 = map_at10 / len(test_grouped)
print(map_at10)

/home/kinetik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':



0.003986710963455149


Мы видим, что качество просело, это объясняется тем, что мы начали рекомендовать из всего пула фильмов (не все видело наше ранжирование), данное упражнение должно было показать вам что такое многостадийные рекомендации. Вы должны усвоить, что обе стадии - ранжирование и кандидатогенерация тесно связаны между собой и очень важны каждая сама по себе.